In [36]:
Position = "31/5/2024"

Location = r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\202405\Python"
File1 = "01. Debtor Listing May 2024 (python)"
File2 = "02. Disbursement & Drawdown May 2024 (python)"
File3 = "03. Stage 3 05 (python)"
File4 = "04. Stage 1 & 2 May24 (python)"
File5 = "05. Data Mirror May 2024 (python)"
File_cont = "06. Contigent Liability May2024 (python)"

#control + F + dropdown new to search bar to find and replace 
import pandas as pd
import numpy as np
import xlsxwriter

#warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

FILE1 = pd.read_excel(str(Location)+"\\"+str(File1)+".xlsx", sheet_name='Export Worksheet', header=0) # skiprows = 2, nrows=18,  usecols= 'A:D'
FILE2 = pd.read_excel(str(Location)+"\\"+str(File2)+".xlsx", sheet_name='Export Worksheet', header=0) # skiprows = 2, nrows=18,  usecols= 'A:D'
FILE3 = pd.read_excel(str(Location)+"\\"+str(File3)+".xlsx", sheet_name='Export Worksheet', header=0) # skiprows = 2, nrows=18,  usecols= 'A:D'
FILE4 = pd.read_excel(str(Location)+"\\"+str(File4)+".xlsx", sheet_name='Export Worksheet', header=0) # skiprows = 2, nrows=18,  usecols= 'A:D'
FILE5 = pd.read_excel(str(Location)+"\\"+str(File5)+".xlsx", sheet_name='Other_Charges_Payment', header=0) # skiprows = 2, nrows=18,  usecols= 'A:D'
FILE6 = pd.read_excel(str(Location)+"\\"+str(File5)+".xlsx", sheet_name='Profit_Payment', header=0) # skiprows = 2, nrows=18,  usecols= 'A:D'
FILE6_1 = pd.read_excel(str(Location)+"\\"+str(File5)+".xlsx", sheet_name='Income in Suspense', header=0) # skiprows = 2, nrows=18,  usecols= 'A:D'

Contigent = pd.read_excel(str(Location)+"\\"+str(File_cont)+".xlsx", sheet_name='BG & LC', header=0) # skiprows = 2, nrows=18,  usecols= 'A:D'

LDB_prev = pd.read_excel(r"C:\Users\syahidhalid\Syahid_PC\Analytics - SPD\Loan Database\LBD 202412 - 202401 (CSP Sector).xlsx", sheet_name='Apr24', header=0) # skiprows = 2, nrows=18,  usecols= 'A:D'

MRate = pd.read_excel(r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\202405\Python\00. Month End Rate.xlsx", header=2) # skiprows = 2, nrows=18,  usecols= 'A:D'

#LDB_prev.columns = LDB_prev.columns.str.replace("\n", "_")
#LDB_prev.columns = LDB_prev.columns.str.replace(" ", "_")
#LDB_prev.columns = LDB_prev.columns.str.replace(".", "")

In [37]:
#+++++++++++++++++++++++++++++++ Debtor Lisiting + Disbuse & Drawdown ================================================================

FILE1['Finance(SAP) Number'] = FILE1['Finance(SAP) Number'].astype(int)
FILE2['Account'] = FILE2['Account'].astype(int)

merge1 = FILE1.drop('Profit Payment/Interest Repayment (MYR)',axis=1).merge(FILE2.rename(columns={'Account':'Finance(SAP) Number','Curr.':'Facility Currency','Type_of_Financing':'Type of Financing'}),
                     on=['Finance(SAP) Number','Facility Currency','Type of Financing'], how='outer')

#check disbruse drawdown & repayment

#print(sum(FILE1['Disbursement/Drawdown (MYR)']))
#print(sum(FILE2.Disbursement_Drawdown_MYR))

#print(sum(FILE1['Cost Payment/Principal Repayment (MYR)']))
#print(sum(FILE2.Cost_Payment_Principal_Repayment_MYR))

In [38]:
#+++++++++++++++++++++++++++++++ ECL Stage 1 2 3 ================================================================

#FILE3['Account_No'] = FILE3['Account_No'].astype(int)
#FILE4['Account_No'] = FILE4['Account_No'].astype(int)

ecl1 = pd.concat([FILE3,FILE4])

ecl1 = ecl1.fillna(0).groupby(['Account_No','Borrower_name','Category','Month'])[['LAF_ECL_FC','LAF_ECL_MYR','CnC_ECL_FC','CnC_ECL_MYR']].sum().reset_index()

#check ecl

#print(FILE3.shape)
#print(FILE4.shape)
#print(ecl1.shape)

In [39]:
#+++++++++++++++++++++++++++++++ Debtor Lisiting + Disbuse & Drawdown + ECL ================================================================

merge1['Finance(SAP) Number'] = merge1['Finance(SAP) Number'].astype(str)
ecl1['Account_No'] = ecl1['Account_No'].astype(str)

merge2 = merge1.merge(ecl1.rename(columns={'Account_No':'Finance(SAP) Number','Month':'Facility Currency','Category':'Type of Financing'}),
                      on=['Finance(SAP) Number','Type of Financing'],how='outer', suffixes=('', '_y'))

merge2.fillna(0, inplace=True)

merge2.loc[merge2['Customer Name'] == 0, 'Customer Name'] = merge2['Borrower_name']
merge2.loc[merge2['Type of Financing'] == 0, 'Type of Financing'] = merge2['Facility Currency_y']

merge2a = merge2.groupby(['Finance(SAP) Number','Facility Currency','Type of Financing','Customer Name'])[[	
                         'Cost/Principal Outstanding (Facility Currency)',	
                         'Cost/Principal Outstanding (MYR)',
                         'Accrued Profit/Interest of the month (Facility Currency)',
                         'Accrued Profit/Interest of the month (MYR)',	
                         'Modification of Loss (Facility Currency)',
                         'Modification of Loss (MYR)',
                         'Cumulative Accrued Profit/Interest (Facility Currency)',
                         'Cumulative Accrued Profit/Interest (MYR)',	
                         'Income/Interest in Suspense (Facility Currency)',
                         'Income/Interest in Suspense (MYR)',
                         'Other Charges (Facility Currency)',
                         'Other Charges (MYR)',		
                         'Total Loans Outstanding (Facility Currency)',	
                         'Total Loans Outstanding (MYR)',	
                         'LAF_ECL_FC',	
                         'LAF_ECL_MYR',	
                         'CnC_ECL_FC',	
                         'CnC_ECL_MYR',		
                         'Disbursement_Drawdown_Facility_Currency',			
                         'Disbursement/Drawdown (MYR)',
                         'Disbursement_Drawdown_MYR',
                         'Cost_Payment_Principal_Repayment_Facility_Currency',
                         'Cost Payment/Principal Repayment (MYR)',	
                         'Cost_Payment_Principal_Repayment_MYR',	
                         ]].sum().reset_index() # 'Profit Payment/Interest Repayment (MYR)',

merge2a1 = merge2a.groupby(['Finance(SAP) Number'])[[	
                         'Cost/Principal Outstanding (Facility Currency)',	
                         'Cost/Principal Outstanding (MYR)',
                         'Accrued Profit/Interest of the month (Facility Currency)',
                         'Accrued Profit/Interest of the month (MYR)',	
                         'Modification of Loss (Facility Currency)',
                         'Modification of Loss (MYR)',
                         'Cumulative Accrued Profit/Interest (Facility Currency)',
                         'Cumulative Accrued Profit/Interest (MYR)',	
                         'Income/Interest in Suspense (Facility Currency)',
                         'Income/Interest in Suspense (MYR)',
                         'Other Charges (Facility Currency)',
                         'Other Charges (MYR)',		
                         'Total Loans Outstanding (Facility Currency)',	
                         'Total Loans Outstanding (MYR)',	
                         'LAF_ECL_FC',	
                         'LAF_ECL_MYR',	
                         'CnC_ECL_FC',	
                         'CnC_ECL_MYR',		
                         'Disbursement_Drawdown_Facility_Currency',			
                         'Disbursement/Drawdown (MYR)',
                         'Disbursement_Drawdown_MYR',
                         'Cost_Payment_Principal_Repayment_Facility_Currency',
                         'Cost Payment/Principal Repayment (MYR)',	
                         'Cost_Payment_Principal_Repayment_MYR',
                         ]].sum().reset_index() # 'Profit Payment/Interest Repayment (MYR)',	

merge2b = merge2a1.merge(merge2a[['Finance(SAP) Number','Facility Currency','Type of Financing','Customer Name']].drop_duplicates('Finance(SAP) Number',keep='first'), on='Finance(SAP) Number',how='left')

#check disbruse drawdown & repayment

#print(sum(merge2b['Disbursement/Drawdown (MYR)']))
#print(sum(merge2b.Disbursement_Drawdown_MYR))

#print(sum(merge2b['Cost Payment/Principal Repayment (MYR)']))
#print(sum(merge2b.Cost_Payment_Principal_Repayment_MYR))

#merge2b.iloc[np.where(merge2b['Finance(SAP) Number']=='501070')]

In [40]:
#+++++++++++++++++++++++++++++++ Other Charges + Profit Payment + Income in Suspense ================================================================

FILE5['Account'] = FILE5['Account'].astype(str)
FILE6['Account'] = FILE6['Account'].astype(str)
FILE6_1['Account'] = FILE6_1['Account'].astype(str)

merge3 = merge2b.merge(FILE5.rename(columns={'Account':'Finance(SAP) Number','Type_of_Financing':'Type of Financing'}),on=['Finance(SAP) Number','Type of Financing'],how='outer')
merge3.fillna(0, inplace=True)

merge4 = merge3.merge(FILE6.rename(columns={'Account':'Finance(SAP) Number','Type_of_Financing':'Type of Financing'}),on=['Finance(SAP) Number','Type of Financing'],how='outer')
merge4.fillna(0, inplace=True)

merge5 = merge4.merge(FILE6_1.rename(columns={'Account':'Finance(SAP) Number','Type_of_Financing':'Type of Financing'}),on=['Finance(SAP) Number','Type of Financing'],how='outer')
merge5.fillna(0, inplace=True)

#print(merge5['Finance(SAP) Number'].value_counts())


In [41]:
merge5a = merge5.groupby(['Finance(SAP) Number','Facility Currency','Type of Financing','Customer Name'])[[	
                         'Cost/Principal Outstanding (Facility Currency)',	
                         'Cost/Principal Outstanding (MYR)',
                         'Accrued Profit/Interest of the month (Facility Currency)',
                         'Accrued Profit/Interest of the month (MYR)',	
                         'Modification of Loss (Facility Currency)',
                         'Modification of Loss (MYR)',
                         'Cumulative Accrued Profit/Interest (Facility Currency)',
                         'Cumulative Accrued Profit/Interest (MYR)',	
                         'Income/Interest in Suspense (Facility Currency)',
                         'Income/Interest in Suspense (MYR)',
                         'Income_Interest_in_Suspense_FC',
                         'Income_Interest_in_Suspense_MYR',
                         'Other Charges (Facility Currency)',
                         'Other Charges (MYR)',		
                         'Total Loans Outstanding (Facility Currency)',	
                         'Total Loans Outstanding (MYR)',	
                         'LAF_ECL_FC',	
                         'LAF_ECL_MYR',	
                         'CnC_ECL_FC',	
                         'CnC_ECL_MYR',		
                         'Disbursement_Drawdown_Facility_Currency',			
                         'Disbursement/Drawdown (MYR)',
                         'Disbursement_Drawdown_MYR',
                         'Cost_Payment_Principal_Repayment_Facility_Currency',
                         'Cost Payment/Principal Repayment (MYR)',	
                         'Cost_Payment_Principal_Repayment_MYR',
                         'Profit_Payment_Interest_Repayment_FC',
                         'Profit_Payment_Interest_Repayment_MYR',
                         'Other_Charges_Payment_FC','Other_Charges_Payment_MYR'
                         ]].sum().reset_index()
 
                         #'Profit_Payment_Interest_Repayment_FC','Profit Payment/Interest Repayment (MYR)',

##cek profit payment

#print(sum(merge4a['Profit Payment/Interest Repayment (MYR)']))
#print(sum(merge3['Profit Payment/Interest Repayment (MYR)']))
#print(sum(FILE6.Profit_Payment_Interest_Repayment_MYR))

In [47]:
#=======================================================LDB Account==================================================

LDB_prev['Finance(SAP) Number'] = LDB_prev['Finance(SAP) Number'].astype(str)
merge5a['Finance(SAP) Number'] = merge5a['Finance(SAP) Number'].astype(str)

merge_ldb = merge5a.merge(LDB_prev[['EXIM Account No.','CIF Number','Finance(SAP) Number','Country Exposure','Facility Currency']].drop_duplicates('Finance(SAP) Number',keep='first'),on=['Finance(SAP) Number'],how='left', suffixes=('', '_y'))
#.rename(columns={'Finance(SAP)_Number':'Finance(SAP) Number'})
#, indicator=True
merge_ldb.sort_values('Total Loans Outstanding (MYR)', ascending=False, inplace=True)

merge_ldb.loc[merge_ldb['Facility Currency']==0, 'Facility Currency'] = merge_ldb['Facility Currency_y']

merge_ldb = merge_ldb.drop('Facility Currency_y',axis=1)
#check

In [48]:
#=======================================================Contigent==================================================

Contigent.loc[Contigent['Type of Financing']=='C', 'Type of Financing'] = "Conventional"
Contigent.loc[Contigent['Type of Financing']=='I', 'Type of Financing'] = "Islamic"

Contigent1 = Contigent.groupby(['EXIM Account No.','CIF Number','Finance(SAP) Number','Customer Name','Type of Financing'])[['Contingent Liability (MYR)','Unutilised/Undrawn Amount (FC)','Unutilised/Undrawn Amount (MYR)','Contingent Liability Letter of Credit (MYR)']].sum().reset_index()

Cont_test = Contigent.sort_values(by='Unutilised/Undrawn Amount (MYR)',ascending=False)[['EXIM Account No.','Country Exposure','Facility Currency']].drop_duplicates('EXIM Account No.',keep='first')

Contigent2= Contigent1.merge(Cont_test,on='EXIM Account No.',how='left')

merge_ldb['EXIM Account No.'] = merge_ldb['EXIM Account No.'].astype(str)
Contigent2['EXIM Account No.'] = Contigent2['EXIM Account No.'].astype(str)

merge_cont = merge_ldb.merge(Contigent2,on=['EXIM Account No.'],how='outer',suffixes=('', '_y'))
#, indicator=True
#.rename(columns={'EXIM Account No.':'EXIM_Account_No'})


In [49]:
merge_cont.loc[merge_cont['Finance(SAP) Number'].isna(), 'Finance(SAP) Number'] = merge_cont['Finance(SAP) Number_y']
merge_cont.loc[merge_cont['Facility Currency'].isna(), 'Facility Currency'] = merge_cont['Facility Currency_y']
merge_cont.loc[merge_cont['Type of Financing'].isna(), 'Type of Financing'] = merge_cont['Type of Financing_y']
merge_cont.loc[merge_cont['Customer Name'].isna(), 'Customer Name'] = merge_cont['Customer Name_y']
merge_cont.loc[merge_cont['CIF Number'].isna(), 'CIF Number'] = merge_cont['CIF Number_y']
merge_cont.loc[merge_cont['Country Exposure'].isna(), 'Country Exposure'] = merge_cont['Country Exposure_y']

merge_cont = merge_cont.fillna(0).drop(['Finance(SAP) Number_y','Facility Currency_y','Type of Financing_y','Customer Name_y','CIF Number_y','Country Exposure_y'],axis=1)

merge_cont.sort_values(by='Total Loans Outstanding (MYR)',ascending=False, inplace=True)

#merge_cont['Finance(SAP) Number'].value_counts()
#merge_cont.iloc[np.where(merge_cont['Finance(SAP) Number']=="BG")]

In [50]:
merge_cont_FC = merge_cont.merge(MRate[['Month','Curr']].rename(columns={'Month':'Facility Currency'}),on='Facility Currency',how='left')

merge_cont_FC['Contingent Liability Letter of Credit (FC)'] = merge_cont_FC['Contingent Liability Letter of Credit (MYR)']*merge_cont_FC['Curr']
merge_cont_FC['Contingent Liability (FC)'] = merge_cont_FC['Contingent Liability (MYR)']*merge_cont_FC['Curr']

merge_cont_FC['Contingent Liability Letter of Credit (FC)'].fillna(0,inplace=True)
merge_cont_FC['Contingent Liability (FC)'].fillna(0,inplace=True)

In [51]:
#=======================================================LDB All==================================================

#email staked holder 1st cut finance risk field required

LDB_prev.columns = LDB_prev.columns.str.strip()
LDB_prev.columns = LDB_prev.columns.str.replace("\n", "")
#LDB_prev.columns = LDB_prev.columns.str.replace(" ", "_")

In [ ]:
#LDB_prev.head(1)

In [ ]:
#LDB_prev['Ta`widh Payment/Penalty Repayment  (MYR)']
#LDB_prev['Cumulative Ta`widh Payment/Penalty Repayment  (MYR)']

In [52]:
LDB_prev1 = LDB_prev[['Finance(SAP) Number','EXIM Account No.','CIF Number','Customer Name','Facility Currency',
                      'Type of Financing','Country Exposure',
                            'Company Group',
                            'Relationship Manager (RM)',
                            'Banking Team',
                            'Ownership',
                            'Officer in Charge',
                            'Syndicated / Club Deal',
                            'Nature of Account',
                            'Facility',
                            'Shariah Contract / Concept',
                            'Fund / Guarantee /Program / Insentif',
                            'Status',
                            'Post Approval Stage',
                            'Date of Ready for Utilization (RU)',
                            'Restructured / Rescheduled(Y/N)',
                            'Amount Approved / Facility Limit (Facility Currency)',
                            'Amount Approved / Facility Limit (MYR)',
                            'Account Receivables/Past Due Claims (Facility Currency)',
                            'Account Receivable/Past Due Claims (MYR)',
                            'Total Banking Exposure (Facility Currency)',
                            'Total Banking Exposure (MYR)',
                            'Penalty/Ta`widh (Facility Currency)',
                            'Penalty/Ta`widh (MYR)',
                            'Disbursement/Drawdown Status',
                            'Cumulative Disbursement/Drawdown (Facility Currency)',
                            'Cumulative Disbursement/Drawdown (MYR)',
                            'Cumulative Cost Payment/Principal Repayment (Facility Currency)',
                            'Cumulative Cost Payment/Principal Repayment (MYR)',
                            'Cumulative Profit Payment/Interest Repayment (Facility Currency)',
                            'Cumulative Profit Payment/Interest Repayment (MYR)',
                            'Ta`widh Payment/Penalty Repayment (Facility Currency)',
                            'Ta`widh Payment/Penalty Repayment  (MYR)',
                            'Cumulative Ta`widh Payment/Penalty Repayment (Facility Currency)',
                            'Cumulative Ta`widh Payment/Penalty Repayment  (MYR)',
                            'Cumulative Other Charges Payment (Facility Currency)',
                            'Cumulative Other Charges Payment (MYR)',
                            'Rating at Origination',
                            'PD',
                            'PF',
                            'CRMS Obligor Risk Rating',
                            'CRMS CG Rating',
                            'PD (%)',
                            'LGD (%)',
                            'Risk Analyst',
                            'MFRS9 Staging',
                            'BNM Main Sector',
                            'BNM Sub Sector',
                            'EXIM Main Sector',
                            'Industry (Risk)',
                            'Industry Classification',
                            'Purpose of Financing',
                            'Date Approved at Origination',
                            'Approval Authority',
                            'LO issuance Date',
                            'Date of LO Acceptance',
                            '1st Disbursement/Drawdown Date',
                            '1st Payment/Repayment Date',
                            'Expiry of Availability Period',
                            'Facility Agreement Date',
                            'Annual Review Date',
                            'Watchlist Review Date',
                            'Maturity/Expired Date',
                            'Grace Period (Month)',
                            'Moratorium Period (Month)',
                            'Start Moratorium Date',
                            'Tenure (Month)',
                            'Payment/Repayment Frequency (Profit/Interest)',
                            'Payment/Repayment Frequency (Cost/Principal)',
                            'Effective cost of borrowings',
                            'Profit/Interest Margin',
                            'Average Profit/Interest Rate',
                            'Ta`widh Compensation/Penalty Rate',
                            'Operation Country',
                            'Country Rating',
                            'Region',
                            'Market Type',
                            'Classification of Entity / Customer Type',
                            'Entity / Customer Type',
                            'Classification of Residency Status',
                            'Residency Status',
                            'Corporate Type',
                            'SME Commercial Corporate',
                            'Corporate Status',
                            'Justification on Corporate Status',
                            'Restructured / Rescheduled',
                            'Date of Approval Restructured / Rescheduled',
                            'Effective Date',
                            'Reason',
                            'Date Untagged from R&R',
                            'Justification for Untagged',
                            'Frequency',
                            'Date of Overdue',
                            'Date Classified as Watchlist',
                            'Watchlist Reason',
                            'Date Declassified from Watchlist',
                            'Date Impaired',
                            'Reason for Impairment',
                            'Partial Write off Date',
                            'Write off Date',
                            'Cancellation Date/Fully Settled Date']].drop_duplicates('Finance(SAP) Number', keep='first')


In [53]:
merge_final = merge_cont_FC.merge(LDB_prev1,on=['Finance(SAP) Number','EXIM Account No.'],how='outer', suffixes=('', '_y'))

merge_final.sort_values('Total Loans Outstanding (MYR)', ascending=False, inplace=True)

merge_final['Position as At'] = Position

merge_final.loc[merge_final['Facility Currency'].isna(), 'Facility Currency'] = merge_final['Facility Currency_y']
merge_final.loc[merge_final['Type of Financing'].isna(), 'Type of Financing'] = merge_final['Type of Financing_y']
merge_final.loc[merge_final['Customer Name'].isna(), 'Customer Name'] = merge_final['Customer Name_y']
merge_final.loc[merge_final['CIF Number'].isna(), 'CIF Number'] = merge_final['CIF Number_y']
merge_final.loc[merge_final['Country Exposure'].isna(), 'Country Exposure'] = merge_final['Country Exposure_y']

merge_final['Cumulative Disbursement/Drawdown (Facility Currency)'] = merge_final['Cumulative Disbursement/Drawdown (Facility Currency)'] + merge_final['Disbursement_Drawdown_Facility_Currency']
merge_final['Cumulative Disbursement/Drawdown (MYR)'] = merge_final['Cumulative Disbursement/Drawdown (MYR)'] + merge_final['Disbursement/Drawdown (MYR)']
merge_final['Cumulative Cost Payment/Principal Repayment (Facility Currency)'] = merge_final['Cumulative Cost Payment/Principal Repayment (Facility Currency)'] + merge_final['Cost_Payment_Principal_Repayment_Facility_Currency']
merge_final['Cumulative Cost Payment/Principal Repayment (MYR)'] = merge_final['Cumulative Cost Payment/Principal Repayment (MYR)'] +  merge_final['Cost Payment/Principal Repayment (MYR)'] 
merge_final['Cumulative Other Charges Payment (Facility Currency)'] = merge_final['Cumulative Other Charges Payment (Facility Currency)'] + merge_final['Other_Charges_Payment_FC']
merge_final['Cumulative Other Charges Payment (MYR)'] = merge_final['Cumulative Other Charges Payment (MYR)'] + merge_final['Other_Charges_Payment_MYR']
merge_final['Cumulative Profit Payment/Interest Repayment (Facility Currency)'] = merge_final['Profit_Payment_Interest_Repayment_FC'] + merge_final['Cumulative Profit Payment/Interest Repayment (Facility Currency)']
merge_final['Cumulative Profit Payment/Interest Repayment (MYR)'] = merge_final['Profit_Payment_Interest_Repayment_MYR'] + merge_final['Cumulative Profit Payment/Interest Repayment (MYR)']


In [54]:
#.rename(columns={'Finance(SAP)_Number':'Finance(SAP) Number'})
#, indicator=True

                            #'Finance(SAP)_Number',
                            #'CIF_Number',
                            #'EXIM_Account_No',
                            #'Status',
                            #'Customer_Name',
                            #'Syndicated_/_Club_Deal',
                            #'Nature_of_Account',
                            #'Facility',
                            #'Facility_Currency',
                            #'Type_of_Financing',
                            #'Amount_Approved_/_Facility_Limit_(Facility_Currency)',
                            #'Amount_Approved_/_Facility_Limit_(MYR)',
                            #'Account_Receivables/Past_Due_Claims_(Facility_Currency)',
                            #'Account_Receivable/Past_Due_Claims_(MYR)',
                            #'Penalty/Ta`widh_(Facility_Currency)',
                            #'Penalty/Ta`widh_(MYR)','Disbursement/_Drawdown_Status',
                            #'Rating_at_Origination',
                            #'PD',
                            #'PF',
                            #'CRMS_Obligor_Risk_Rating',
                            #'CRMS_CG_Rating',
                            #'Risk_Analyst',
                            #'MFRS9_Staging',
                            #'BNM_Main_Sector',
                            #'BNM_Sub_Sector',
                            #'EXIM_Main_Sector',
                            #'Industry_(Risk)',
                            #'Industry_Classification',
                            #'Purpose_of_Financing',
                            #'Date_Approved_at_Origination',
                            #'Approval_Authority'

merge_final = merge_final.drop(['Facility Currency_y','Type of Financing_y','Customer Name_y','CIF Number_y','Country Exposure_y'],axis=1)

merge_final['Cost/Principal Outstanding (Facility Currency)'].fillna(0, inplace=True)
merge_final['Cost/Principal Outstanding (MYR)'].fillna(0, inplace=True)
merge_final['Accrued Profit/Interest of the month (Facility Currency)'].fillna(0, inplace=True)
merge_final['Accrued Profit/Interest of the month (MYR)'].fillna(0, inplace=True)
merge_final['Modification of Loss (Facility Currency)'].fillna(0, inplace=True)
merge_final['Modification of Loss (MYR)'].fillna(0, inplace=True)
merge_final['Cumulative Accrued Profit/Interest (Facility Currency)'].fillna(0, inplace=True)
merge_final['Cumulative Accrued Profit/Interest (MYR)'].fillna(0, inplace=True)
merge_final['Income/Interest in Suspense (Facility Currency)'].fillna(0, inplace=True)
merge_final['Income/Interest in Suspense (MYR)'].fillna(0, inplace=True)
merge_final['Income_Interest_in_Suspense_FC'].fillna(0, inplace=True)
merge_final['Income_Interest_in_Suspense_MYR'].fillna(0, inplace=True)
merge_final['Other Charges (Facility Currency)'].fillna(0, inplace=True)
merge_final['Other Charges (MYR)'].fillna(0, inplace=True)
merge_final['Total Loans Outstanding (Facility Currency)'].fillna(0, inplace=True)
merge_final['Total Loans Outstanding (MYR)'].fillna(0, inplace=True)
merge_final['LAF_ECL_FC'].fillna(0, inplace=True)
merge_final['LAF_ECL_MYR'].fillna(0, inplace=True)
merge_final['CnC_ECL_FC'].fillna(0, inplace=True)
merge_final['CnC_ECL_MYR'].fillna(0, inplace=True)
merge_final['Disbursement_Drawdown_Facility_Currency'].fillna(0, inplace=True)
merge_final['Disbursement/Drawdown (MYR)'].fillna(0, inplace=True)
merge_final['Disbursement_Drawdown_MYR'].fillna(0, inplace=True)
merge_final['Cost_Payment_Principal_Repayment_Facility_Currency'].fillna(0, inplace=True)
merge_final['Cost Payment/Principal Repayment (MYR)'].fillna(0, inplace=True)
merge_final['Cost_Payment_Principal_Repayment_MYR'].fillna(0, inplace=True)
merge_final['Profit_Payment_Interest_Repayment_FC'].fillna(0, inplace=True)
merge_final['Profit_Payment_Interest_Repayment_MYR'].fillna(0, inplace=True)
merge_final['Profit_Payment_Interest_Repayment_FC'].fillna(0, inplace=True)
merge_final['Profit_Payment_Interest_Repayment_MYR'].fillna(0, inplace=True)
merge_final['Other_Charges_Payment_FC'].fillna(0, inplace=True)
merge_final['Other_Charges_Payment_MYR'].fillna(0, inplace=True)
merge_final['Cumulative Disbursement/Drawdown (Facility Currency)'].fillna(0, inplace=True)
merge_final['Cumulative Disbursement/Drawdown (MYR)'].fillna(0, inplace=True)
merge_final['Cumulative Cost Payment/Principal Repayment (Facility Currency)'].fillna(0, inplace=True)
merge_final['Cumulative Cost Payment/Principal Repayment (MYR)'].fillna(0, inplace=True) 
merge_final['Cumulative Other Charges Payment (Facility Currency)'].fillna(0, inplace=True)
merge_final['Cumulative Other Charges Payment (MYR)'].fillna(0, inplace=True)
merge_final['Cumulative Profit Payment/Interest Repayment (Facility Currency)'].fillna(0, inplace=True)
merge_final['Cumulative Profit Payment/Interest Repayment (MYR)'].fillna(0, inplace=True)
merge_final['Ta`widh Payment/Penalty Repayment (Facility Currency)'].fillna(0, inplace=True)
merge_final['Ta`widh Payment/Penalty Repayment  (MYR)'].fillna(0, inplace=True)
merge_final['Cumulative Ta`widh Payment/Penalty Repayment (Facility Currency)'].fillna(0, inplace=True)
merge_final['Cumulative Ta`widh Payment/Penalty Repayment  (MYR)'].fillna(0, inplace=True)

merge_final.sort_values('Total Loans Outstanding (MYR)', ascending=False, inplace=True)

merge_final = merge_final.drop(['Income_Interest_in_Suspense_FC',
                                'Income_Interest_in_Suspense_MYR',
                                'Disbursement_Drawdown_MYR',
                                'Cost_Payment_Principal_Repayment_MYR'],axis=1)


                               # 'Profit_Payment_Interest_Repayment_FC',
                                # 'Profit_Payment_Interest_Repayment_MYR'


In [ ]:
#merge_final['Finance(SAP) Number'].value_counts()

In [ ]:
#merge_final.iloc[np.where(merge_final['Finance(SAP) Number'].str.contains("BG"))]['Finance(SAP) Number'].value_counts()

In [ ]:
#merge_final.tail(1)

In [55]:
merge_final1 = merge_final[['CIF Number','EXIM Account No.','Finance(SAP) Number',
                            'Company Group',
                            'Customer Name',
                            'Relationship Manager (RM)',
                            'Banking Team',
                            'Ownership',
                            'Officer in Charge',
                            'Syndicated / Club Deal',
                            'Nature of Account',
                            'Facility',
                            'Facility Currency',
                            'Type of Financing',
                            'Shariah Contract / Concept',
                            'Fund / Guarantee /Program / Insentif',
                            'Status',
                            'Post Approval Stage',
                            'Date of Ready for Utilization (RU)',
                            'Restructured / Rescheduled(Y/N)',
                            'Amount Approved / Facility Limit (Facility Currency)',
                            'Amount Approved / Facility Limit (MYR)',
                            'Cost/Principal Outstanding (Facility Currency)',
                            'Cost/Principal Outstanding (MYR)',
                            'Contingent Liability Letter of Credit (FC)',
                            'Contingent Liability Letter of Credit (MYR)',
                            'Contingent Liability (FC)',
                            'Contingent Liability (MYR)',
                            'Account Receivables/Past Due Claims (Facility Currency)',
                            'Account Receivable/Past Due Claims (MYR)',
                            'Total Banking Exposure (Facility Currency)',
                            'Total Banking Exposure (MYR)',
                            'Accrued Profit/Interest of the month (Facility Currency)',
                            'Accrued Profit/Interest of the month (MYR)',
                            'Modification of Loss (Facility Currency)',
                            'Modification of Loss (MYR)',
                            'Cumulative Accrued Profit/Interest (Facility Currency)',
                            'Cumulative Accrued Profit/Interest (MYR)',                            
                            'Penalty/Ta`widh (Facility Currency)',
                            'Penalty/Ta`widh (MYR)',
                            'Income/Interest in Suspense (Facility Currency)',
                            'Income/Interest in Suspense (MYR)',
                            'Other Charges (Facility Currency)',
                            'Other Charges (MYR)',
                            'Total Loans Outstanding (Facility Currency)',
                            'Total Loans Outstanding (MYR)',
                            'LAF_ECL_FC',
                            'LAF_ECL_MYR',
                            'CnC_ECL_FC',
                            'CnC_ECL_MYR',
                            'Disbursement/Drawdown Status',
                            'Unutilised/Undrawn Amount (FC)',
                            'Unutilised/Undrawn Amount (MYR)',
                            'Disbursement_Drawdown_Facility_Currency',
                            'Disbursement/Drawdown (MYR)',
                            'Cumulative Disbursement/Drawdown (Facility Currency)',
                            'Cumulative Disbursement/Drawdown (MYR)',
                            'Cost_Payment_Principal_Repayment_Facility_Currency',
                            'Cost Payment/Principal Repayment (MYR)',                            
                            'Cumulative Cost Payment/Principal Repayment (Facility Currency)',
                            'Cumulative Cost Payment/Principal Repayment (MYR)',
                            'Profit_Payment_Interest_Repayment_FC',
                            'Profit_Payment_Interest_Repayment_MYR',
                            'Cumulative Profit Payment/Interest Repayment (Facility Currency)',
                            'Cumulative Profit Payment/Interest Repayment (MYR)',
                            'Ta`widh Payment/Penalty Repayment (Facility Currency)',
                            'Ta`widh Payment/Penalty Repayment  (MYR)',
                            'Cumulative Ta`widh Payment/Penalty Repayment (Facility Currency)',
                            'Cumulative Ta`widh Payment/Penalty Repayment  (MYR)',
                            'Other_Charges_Payment_FC',
                            'Other_Charges_Payment_MYR',
                            'Cumulative Other Charges Payment (Facility Currency)',
                            'Cumulative Other Charges Payment (MYR)',
                            'Rating at Origination',
                            'PD',
                            'PF',
                            'CRMS Obligor Risk Rating',
                            'CRMS CG Rating',
                            'PD (%)',
                            'LGD (%)',
                            'Risk Analyst',
                            'MFRS9 Staging',
                            'BNM Main Sector',
                            'BNM Sub Sector',
                            'EXIM Main Sector',
                            'Industry (Risk)',
                            'Industry Classification',
                            'Purpose of Financing',
                            'Date Approved at Origination',
                            'Approval Authority',
                            'LO issuance Date',
                            'Date of LO Acceptance',
                            '1st Disbursement/Drawdown Date',
                            '1st Payment/Repayment Date',
                            'Expiry of Availability Period',
                            'Facility Agreement Date',
                            'Annual Review Date',
                            'Watchlist Review Date',
                            'Maturity/Expired Date',
                            'Grace Period (Month)',
                            'Moratorium Period (Month)',
                            'Start Moratorium Date',
                            'Tenure (Month)',
                            'Payment/Repayment Frequency (Profit/Interest)',
                            'Payment/Repayment Frequency (Cost/Principal)',
                            'Effective cost of borrowings',
                            'Profit/Interest Margin',
                            'Average Profit/Interest Rate',
                            'Ta`widh Compensation/Penalty Rate',
                            'Operation Country',
                            'Country Exposure',
                            'Country Rating',
                            'Region',
                            'Market Type',
                            'Classification of Entity / Customer Type',
                            'Entity / Customer Type',
                            'Classification of Residency Status',
                            'Residency Status',
                            'Corporate Type',
                            'SME Commercial Corporate',
                            'Corporate Status',
                            'Justification on Corporate Status',
                            'Restructured / Rescheduled',
                            'Date of Approval Restructured / Rescheduled',
                            'Effective Date',
                            'Reason',
                            'Date Untagged from R&R',
                            'Justification for Untagged',
                            'Frequency',
                            'Date of Overdue',
                            'Date Classified as Watchlist',
                            'Watchlist Reason',
                            'Date Declassified from Watchlist',
                            'Date Impaired',
                            'Reason for Impairment',
                            'Partial Write off Date',
                            'Write off Date',
                            'Cancellation Date/Fully Settled Date','Position as At']]

In [56]:
merge_final1.to_excel(str(Location)+'\\Testing v3 _ Loan Database as '+str(File1[-17:])+'.xlsx', sheet_name='Export Worksheet', index = False)
